In [35]:
import re
import pandas as pd
from sklearn import linear_model

# Chapter 3

### Exercise 8

In [94]:
auto_df = pd.read_csv('./data/auto.csv')
auto_df.info()
auto_df.head(200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           397 non-null    float64
 1   cylinders     397 non-null    int64  
 2   displacement  397 non-null    float64
 3   horsepower    397 non-null    object 
 4   weight        397 non-null    int64  
 5   acceleration  397 non-null    float64
 6   year          397 non-null    int64  
 7   origin        397 non-null    int64  
 8   name          397 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.0+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
195,29.0,4,85.0,52,2035,22.2,76,1,chevrolet chevette
196,24.5,4,98.0,60,2164,22.1,76,1,chevrolet woody
197,29.0,4,90.0,70,1937,14.2,76,2,vw rabbit
198,33.0,4,91.0,53,1795,17.4,76,3,honda civic


In [97]:
auto_df['horsepower'].str.match(r'\d')
auto_df['horsepower'].replace(r'[0-9]', None, regex = False,  inplace = True)
print(auto_df['horsepower'].str.match(r'\d' == False))

TypeError: first argument must be string or compiled pattern